In [2]:
import numpy as np
import tensorflow as tf
from PIL import Image
from pathlib import Path
import pandas as pd

# === 1. Load the TFLite model ===
interpreter = tf.lite.Interpreter(model_path="mobile_TEST.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']  # e.g., (1, 3, 256, 256)
input_dtype = input_details[0]['dtype']

# === 2. Image Preprocessing ===
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((input_shape[2], input_shape[3]))  # Resize to expected input shape (W, H)
    img = np.asarray(img).astype(np.float32)

    # If input shape is (1, 3, 256, 256), transpose
    if input_shape[1] == 3:
        img = np.transpose(img, (2, 0, 1))  # (H, W, C) -> (C, H, W)

    img = img / 255.0  # Normalize if required
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.astype(input_dtype)

In [3]:
# Prediction function
def get_predictions(image_path, threshold=0.85):
    # Preprocess image
    input_data = preprocess_image(image_path)

    # Set tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get raw predictions
    tflite_output = interpreter.get_tensor(output_details[0]['index'])  # NumPy array
    probs = tf.nn.softmax(tflite_output, axis=-1).numpy()
    print(f"Probabilities for {image_path.name}:", probs)

    probabilities = probs[0]  # Remove batch dimension

    # Assume binary classification: class 0 = 'original', class 1 = 'recaptured'
    pred_idx = np.argmax(probabilities)
    pred_class = 'Recaptured' if pred_idx == 1 else 'clear'
    recaptured_probability = float(probabilities[1])  # Confidence for class 1

    flag = 1 if recaptured_probability >= threshold else 0

    return flag, recaptured_probability, pred_class

    # # Get predictions
    # output_data = interpreter.get_tensor(output_details[0]['index'])  # This will be a NumPy array
    # probabilities = output_data[0]  # Remove batch dimension

    # # Assume binary classification: class 0 = 'original', class 1 = 'recaptured'
    # pred_idx = np.argmax(probabilities)
    # pred_class = 'Recaptured' if pred_idx == 1 else 'clear'
    # recaptured_probability = float(probabilities[1])  # Confidence for class 1

    # flag = 1 if recaptured_probability >= threshold else 0

    # return flag, recaptured_probability, pred_class


In [ ]:
# Define base folder with images
base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\NMMS PHOTOS OF 10TH APRIL")

# Get all images (limit to first 5000)
image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))
# image_paths = image_paths[:5]

print(f"Found {len(image_paths)} images. Starting prediction..\n")

# Prediction loop
results = []

for img_path in image_paths:
    try:
        flag, probability, pred_class = get_predictions(img_path)
        results.append({
            'Image': str(img_path),
            'Class': pred_class,
            'Flag': flag,
            'Recaptured_Probability': probability
        })
    except Exception as e:
        print(f"Error with image {img_path.name}: {e}")

# Create DataFrame and save to CSV
df_results = pd.DataFrame(results)
print("\n Sample results:")
print(df_results.head())

In [ ]:
df_results[df_results.Image =='C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\102_1726002003_WC_22012035247758_2025-04-10R1042025111619.jpeg']# 

In [ ]:
# from PIL import Image

# # Open the image
# img = Image.open('C:/Users/arpitasharma2/OneDrive - Deloitte (O365D)/Desktop/DIU/NMMS PHOTOS OF 10TH APRIL/NMMS PHOTOS OF 10TH APRIL/2_1710004075_WC_22012035230693_2025-04-10R1042025043729.jpeg')

# # Optional: Show the image
# # img.show()
# img


In [ ]:
# df_results.shape

In [ ]:
df_results.head()

In [ ]:
df_results.to_csv("test.csv", index=False)
print("\n Results saved to 'test.csv'")

In [ ]:
import shutil
import os

# Define output folders
output_dir = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\flagged_nnms")
flag_0_dir = output_dir / "flag_0_tflite"
flag_1_dir = output_dir / "flag_1_tflite"

# Create directories if they don't exist
flag_0_dir.mkdir(parents=True, exist_ok=True)
flag_1_dir.mkdir(parents=True, exist_ok=True)

# Copy images to respective folders
for _, row in df_results.iterrows():
    src = Path(row['Image'])
    dest_dir = flag_1_dir if row['Flag'] == 1 else flag_0_dir
    try:
        shutil.copy(src, dest_dir)
    except Exception as e:
        print(f" Could not copy {src.name} to {dest_dir}: {e}")

print(f"\n Images have been copied into respective folders:\yu8/>\"n- {flag_0_dir}\n- {flag_1_dir}")
